# **GEOS2115/2915 2023 Practical Weeks 3 and 4:** Historical and future climate change in observations and CMIP6 models.

In weeks 3 and 4 of the practicals for GEOS2115 we will be analysing output from simulations run as part of the World Climate Research Program (WCRP)'s Coupled Model Intercomparison Program (CMIP6). We will compare these simulations to some observations, look at their representation of historical climate change and their projections/predictions of how the climate may change in the future.

This practical is split into two parts:
- Week 3: Introduction to analysing and plotting the CMIP6 data \& historical climate in ACCESS-CM2.
- Week 4: Projected future climate change in CMIP6

This notebook is for Week 3.

# Week 3: Introduction to CMIP6 data and historical climate in ACCESS-CM2.

## The CMIP6 model data and observations

As discussed in the lectures, CMIP6 is a framework for climate modelling groups around the world to design, perform and compare the results of their latest simulations of historical and future climate and climate change. In this practical exercise we will be analysing data from 3 different climate models:
- The [Australian Community Climate and Earth System Simulator Climate Model Version 2.0 (ACCESS-CM2)](https://research.csiro.au/access/about/cm2/). Australia's flagship coupled climate model run by CSIRO.
- The [Max-Planck Institut for Meteorology Earth System Model (MPI-ESM)](https://mpimet.mpg.de/en/science/models/mpi-esm).
- The [Canadian Earth System Model Version 5.0 (CanESM5)](https://gmd.copernicus.org/articles/12/4823/2019/gmd-12-4823-2019.html).

We will be focusing on two different types of simulations performed with these models:
- **Historical simulations**: These simulations correspond to the model's representation of how climate has evolved over the historical period, 1850-2014. 
- **Future projection simulations**: These simulations are estimates of how the climate might change in the future given different scenarios for greenhouse gas emissions known as Shared Socio-Economic pathways (SSPs). You will have access to data from two SSPs; SSP245 and SSP585 (see Week 4 practical for more information), out to the end of the 21st century (year 2100).

Finally, we also provide some observational data with which the model results will be compared (over the historical period). This data is from the European Centre for Medium-Range Weather Forecasts (ECMWF) [20th Century atmospheric reanalysis (ERA20C)](https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era-20c). The ERA20C product spans the period 1900-2010. While ERA20C is a model-derived product, it uses *data assimilation* to include a large range of satellite and in-situ observations and thus corresponds to a best estimate of what the climate system was doing over this time period (as opposed to the historical CMIP6 simulations which do not assimilate observations).

Our analysis will focus on several important physical variables in the climate system including:
- Surface temperature
- Downward and upward longwave and shortwave radiation at the surface and the top of the atmosphere (TOA).
- Sea level
- Precipitation

Data is provided in the form of [netCDF files](https://climatedataguide.ucar.edu/climate-tools/NetCDF), one for each model simulation. You will be given examples of how to load and plot the data that is extracted from these files. You will then be asked to make plots and answer questions based on these plots.

If you are interested in further reading on the CMIP6 simulations a few links are provided here:
1. https://www.carbonbrief.org/cmip6-the-next-generation-of-climate-models-explained/
2. https://opus.nci.org.au/display/CMIP/Overview+on+CMIP
3. https://climate-scenarios.canada.ca/?page=cmip6-overview-notes#shared-socio-economic-pathways(ssps)

### Xarray datasets

We will start out with a short introduction to working with the CMIP6 data. We will use the python package [xarray](https://docs.xarray.dev/en/stable/), which makes it really easy to make plots from netcdf variables in just a few simple commands. First, we load a few python modules, including xarray:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
xr.set_options(keep_attrs=True) # Keep attributes like units through calculations
import cmocean as cm

The netcdf data files are contained in the read-only `/course/data/` folder. We can see the contents of `/course/data/` by executing the following cell:

In [2]:
!ls /course/data/

You should see a list of 13 netcdf (`.nc`) files. The first part of the file name indicates the model (ACCESS-CM2, CanESM5 or MPI-ESM). Two *ensemble members* are provided for the ACCESS-CM2 data (ACCESS-CM2 and ACCESS-CM2-E2). For each model there is a file for data from the historical simulation (hist) and for the ssp245 and ssp585 future projections. Finally, there is also a data file containing the ERA20C observational data (`ERA20C_hist_data.nc`).

Data can be loaded into the jupyter-notebook using a simple command. Here we start by loading the ACCESS-CM2 historical simulation data into an xarray dataset, ACCESS_CM2_hist, and then printing the contents of that dataset:

In [3]:
ACCESS_CM2_hist = xr.open_dataset('/course/data/ACCESS-CM2_hist_data.nc')
ACCESS_CM2_hist

The data consists of a series of *Data variables* most of which are defined at each latitude and longitude covering the globe and vary with year. There are 165 years corresponding to each year between 1850 and 2014.

For each *Data variable*, you can click on the document symbol (![images/Doc.png](images/Doc.png)) to see more information on the variable. Alternatively, you can access the variable itself by using a dot (`.`) and then the variable name. For example for the variable SAT:

In [4]:
ACCESS_CM2_hist.SAT

Reading the variable *attributes* indicates that the SAT variable corresponds to the *Near-Surface Air Temperature* and has units of degrees celsius. This is the surface temperature variable that has been discussed in the Practical Week 1 (`Ts`). Except here it is defined at each latitude and longitude instead of as a global average over the entire planet.

<font color='blue'> Examine the other variables that have been provided in the data file. </font>

### Making plots from Xarray Datasets

Xarray makes it really easy to make simple plots of variables from netcdf data. As an example, in the next code block we make a plot of the surface air temperature (SAT) variable from the first year of the ACCESS-CM2 historical simulation, 1850.

The year is chosen using xarray's select (`.sel`) function.

In [5]:
# Plot ACCESS_CM2_hist SAT in 1850 using a red-blue colormap, a
# minimum temperature of -20C and a maximum temperature of 30C
ACCESS_CM2_hist.SAT.sel(year=1850).plot(cmap='RdBu_r',vmin=-20.,vmax=30.)

# Add a title to the plot:
plt.title('ACCESS-CM2 Historical SAT 1850')

<font color='blue'> Try replotting the above temperature field but for the last year (2014) instead of the first year. Do you notice much difference?   </font>

If we want to have multiple plots in the same figure, we can use the subplots command. The next code block plots the ACCESS-CM2 SAT in 1850, in 2014, and their difference on the same figure. To make things easier, we first define new variables for the SAT in the two years, and then calculate their difference.

In [0]:
# Define new variables for the 1850 and 2014 SAT, and calculate their difference (DIFF):
SAT1850 = ACCESS_CM2_hist.SAT.sel(year=1850)
SAT2014 = ACCESS_CM2_hist.SAT.sel(year=2014)
DIFF = SAT2014-SAT1850

# Define a four-panel figure (2 rows and 2 columns) for plotting the output:
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,11))

# Plot the 1850 SAT in panel 1:
SAT1850.plot(ax=axes[0][0],cmap='RdBu_r',vmin=-20.,vmax=30.)
axes[0][0].set_title('ACCESS-CM2 Historical SAT 1850')

# Plot the 2014 SAT in panel 2:
SAT2014.plot(ax=axes[1][0],cmap='RdBu_r',vmin=-20.,vmax=30.)
axes[1][0].set_title('ACCESS-CM2 Historical SAT 2014')

# Plot the difference between the two in panel 3:
DIFF.plot(ax=axes[0][1],cmap='RdBu_r',vmin=-5.,vmax=5.)
axes[0][1].set_title('ACCESS-CM2 Historical SAT in 2014 minus SAT in 1850')

# Turn off panel 4:
axes[1][1].axis('off')

it's now much easier to see that the `SAT` is warmer in most regions (although not all regions) in 2014 than it was in 1850, according to the ACCESS-CM2 model.

In this week's practical we will focus on spatial plots such as these. Next week, we will start looking at time series to understand in more detail how the climate has changed over time.

# Mean climate in ACCESS-CM2

Now that we now how to work with the data we will begin our analysis by looking at the mean climate and the radiation balance over the historical period from ACCESS-CM2. For this purpose, we start out by taking a time-mean of the ACCESS-CM2 data using xarray's `.mean` function.

In [0]:
# Take mean of the ACCESS-CM2 data across all years in the historical period.
ACCESS_CM2_hist_mean = ACCESS_CM2_hist.mean('year')

# Display the data we now have:
ACCESS_CM2_hist_mean

You can see that now the `year` dimension has disappeared and we have data that is just defined as a function of latitude and longitude, corresponding to the time mean over all years of the original data.

## Shortwave radiation and albedo

As discussed in the lectures, the dominant source of energy driving the Earth's climate system is the shortwave radiation arriving from the sun. In the model data, this is represented by the field `SWdownTOA` (downward shortwave radiation at the top of the atmosphere). A proportion of this incoming shortwave radiation is reflected back to space, represented by the field `SWupTOA` (upward shortwave radiation at the top of the atmosphere). The proportion that is reflected is determined by the albedo. 

The difference between the downward and reflected shortwave radiation corresponds to that which is absorbed, referred to as the Absorbed Shortwave Radiation (ASR).

In the following code blocks, we first calculate and then plot these different quantities from ACCESS-CM2 in a figure with four panels

In [0]:
# Load the SW data and calculate the albedo:
SWdownTOA = ACCESS_CM2_hist_mean.SWdownTOA        # downward SW radiation at TOA
SWupTOA = ACCESS_CM2_hist_mean.SWupTOA            # upward SW radiation at TOA
ASR = SWdownTOA - SWupTOA                         # Absorbed SW radiation
albedo = SWupTOA/SWdownTOA                        # Albedo (ratio of reflected vs. incoming SW radiation)

In [0]:
# Define a four-panel figure (2 rows and 2 columns) for plotting the output:
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,11))

# Plot the downward SW in panel 1:
SWdownTOA.plot(ax=axes[0][0],cmap=cm.cm.amp,vmin=0.,vmax=450.)
axes[0][0].set_title('ACCESS-CM2 TOA downward SW')

# Plot the upward SW in panel 2:
SWupTOA.plot(ax=axes[0][1],cmap=cm.cm.amp,vmin=0.,vmax=200.)
axes[0][1].set_title('ACCESS-CM2 TOA upward SW')

# Plot the ASR in panel 3:
ASR.plot(ax=axes[1][0],cmap=cm.cm.amp,vmin=-400.,vmax=400.)
axes[1][0].set_title('ACCESS-CM2 Absorbed Solar Radiation')

# Plot the albedo in panel 4:
albedo.plot(ax=axes[1][1],cmap=cm.cm.amp,vmin=0.,vmax=1.)
axes[1][1].set_title('ACCESS-CM2 albedo')

<font color='red'> Using the results from the above figure, answer the following questions: </font>

8. <font color='red'> Why is the top-of-the-atmosphere downward SW radiation uniform in longitude, and what explains its latitudinal structure? **[2 marks]** </font>
9. <font color='red'> Where is the albedo maximum and at what values? Why? **[2 marks]** </font>
10. <font color='red'> Where is the albedo minimum and at what values? Why? **[2 marks]** </font>
11. <font color='red'> Where is the ASR maximum and why? **[2 marks]** </font>

## Longwave radiation

As we discussed in the lectures and in the Week 2 practical, at equilibrium and averaged over the entire Earth, the absorbed solar radiation (ASR) is balance by outgoing longwave radiation (OLR) emitted back into space according to the Stefan-Boltzman law. Let's take a look at the longwave radiation fields from ACCESS-CM2.

The OLR in ACCESS-CM2 is given by the variable `LWupTOA`. This is loaded into the `OLR` variable in the next code block.

We will also take a look at the upward and downward longwave radiation at the surface (i.e. between the ocean/land surface and the atmosphere). The downward longwave radiation at the surface is given by the variable `LWdownSFC` and loaded into this variable in the next code block. 

Assessment question/plot:

12. <font color="red"> Add code in place of the `???` in the next code block to load the upward longwave radiation at the surface (`LWupSFC`), and the surface net long-wave radiation (`Ln`, being the upward longwave radiation at the surface minus the downward longwave radiation at the surface). Then modify the plotting code in the next code block to plot these four variables with appropriate colorbar limits (i.e. replace the `vmin` and `vmax` limits given by `???` with appropriate values). Include this plot in your final report. **[2 marks**] </font> 

In [0]:
# Load longwave properties:
OLR = ACCESS_CM2_hist_mean.LWupTOA           # Outgoing longwave radiation (upward longwave radiation at the top-of-atmosphere)
LWdownSFC = ACCESS_CM2_hist_mean.LWdownSFC   # Downward longwave radiation at the surface
LWupSFC = ???                                # Upward longwave radiation at the surface
Ln = ???                                     # Surface net longwave radiation

In [0]:
# Define a four-panel figure (2 rows and 2 columns) for plotting the output:
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,11))

# Plot OLR:
OLR.plot(ax=axes[0][0],vmin=0.,vmax=500.,cmap=cm.cm.amp)
axes[0][0].set_title('ACCESS-CM2 OLR')

# Plot LWdownSFC:
LWdownSFC.plot(ax=axes[0][1],vmin=0.,vmax=500.,cmap=cm.cm.amp)
axes[0][1].set_title('ACCESS-CM2 SFC downward LW')

# Plot LWupSFC:
LWupSFC.plot(ax=axes[1][0],vmin=???,vmax=???,cmap=cm.cm.amp)
axes[1][0].set_title('ACCESS-CM2 SFC upward LW')

# Plot Ln:
Ln.plot(ax=axes[1][1],vmin=???,vmax=???,cmap='RdBu_r')
axes[1][1].set_title('ACCESS-CM2 SFC net LW')

<font color='red'> Using the results from your figure, answer the following questions: </font>

13. <font color='red'> Why is the upward LW radiation at the TOA much smaller than that at the surface? **[2 marks]** </font>
14. <font color='red'> In which regions is the surface net longwave radiation maximum, and why? **[2 marks]** </font>

## Total Absorbed Radiation

The difference between the ASR and the OLR is the total absorbed radiation. In equilibrium and averaged over the entire Earth, this is zero (as the ASR and OLR balance). However, even at equilibrium the ASR and OLR *at any one location* may not balance each other.

Assessment question/plot:

15. <font color="red"> By copying code from above, in the next code block add code to both calculate the total absorbed radiation (`TAR`) from `ASR` and `OLR` and make plots of the ASR, the OLR and the TAR. **[3 marks**] </font>

For example, you could copy the code for the four-panel plot for LW radiation above and change the variables that are plotted, the colorbar limits and possibly the type of colorbar (e.g. the white-to-red colorbar given by `cmap=cm.cm.amp` or the blue-to-red colorbar given by `cmap='RdBu_r'`). You can leave the fourth panel blank.

In [0]:
# Calculate the TAR:
TAR = ???           # Total absorbed radiation

In [0]:
# Define a four-panel figure (2 rows and 2 columns) for plotting the output:
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,11))

# ADD CODE HERE (copy from above and modify) to plot the ASR, OLR and TAR in 
# 3 of the four panels (you can leave the last panel blank) 

Assessment question:

16. <font color="red"> Describe the latitudinal structure of the ASR, the OLR and the TAR. Discuss what these results imply about atmospheric and oceanic circulation and heat transport **[3 marks**] </font>

## Historical climate change in ACCESS-CM2

Now that we have an understanding of the mean climate and radiation balance in ACCESS-CM2, let's have a look at how the climate changes over the historical period. In CMIP6, the historical period is generally classified as 1850-2014. The final year (2014) is the last year for which forcings (e.g. CO2 concentration) were provided for the CMIP experiments. The initial year is chosen as it is before the vast majority of anthropogenic greenhouse emissions.

Our first objective is to plot the evolution of the global mean surface temperature (GMST) over the historical period. In the introduction material above, we showed how we can plot a spatial map of the surface temperature. For example, we can plot a spatial map of the mean `SAT` from ACCESS-CM2:

In [0]:
ACCESS_CM2_hist_mean.SAT.plot(cmap='RdBu_r',vmin=-20.,vmax=30.)
plt.title('ACCESS-CM2 Mean SAT')

We now would like to know how to calculate the global mean of this field.  We could use the Xarray function `mean`, as we did for the time-mean. However, such a mean would not take into account the fact that the size of the grid cells in the model are different at different latitudes. I.e. it would equally weight grid cells such that grid cells that cover only a small area (say right next to the poles) would be equally as important as grid cells that cover a large area. This is illustrated in the following figure:

<div>
<center><img src="images/grid.jpg" width="250"/>
    </center>
    <p style="font-size:10px;">Image credit: https://doi.org/10.1098/rsta.2008.0219</p>
</div>

Instead, we need to do an "area-weighted" mean. This can be done by first multiplying the variable we're interested in by the area in each grid cell (the `area` variable contained without our data files), summing over all cells and then dividing by the total area. As we will use this operation a few times, we define a python function for it in the next code block.

In [0]:
# Define a function to calculate the global average of the variable "variable" 
# using an equal-area weighting according to the "area" dataset
def global_average(variable,area):
    # Calculate the total area:
    total_area = area.where(variable.notnull()).sum('longitude').sum('latitude')
    
    # Calculate the global average by multiplying the variable by the 
    # grid cell area at each grid cell, summing, and then dividing by the total area:
    average = (variable*area).sum('longitude').sum('latitude')/total_area
    
    return(average)

We can now use this function to calculate the global mean SAT:

In [0]:
SAT_global = global_average(ACCESS_CM2_hist_mean.SAT,ACCESS_CM2_hist_mean.area)
print('ACCESS-CM2 GMST = %3.2f' % (SAT_global))

This is pretty close to what we might expect (around 15C).

We can apply the same function to the `ACCESS_CM2_hist.SAT` variable (i.e. without the `_mean`) to make a **time series** of the GMST:

In [0]:
# Calculate the GMST as a function of time:
ACCESS_CM2_GMST = global_average(ACCESS_CM2_hist.SAT,ACCESS_CM2_hist.area)

# Plot it:
ACCESS_CM2_GMST.plot()
plt.title('Global Mean Surface Temperature')

This plot shows a strong warming of GMST since the 1970s and 1980s in the model.

How does this historical climate change compare to observations? To answer this question we can compare the model results to the observational ERA20C data provided in the `/course/data/ERA20C_hist_data.nc` file. The following code block loads this data, calculates the GMST and then plots it along with the ACCESS-CM2 data:

In [0]:
# Load ERA20C data and calculate GMST:
ERA20C_hist = xr.open_dataset('/course/data/ERA20C_hist_data.nc')
ERA20C_GMST = global_average(ERA20C_hist.SAT,ERA20C_hist.area)

# Plot both the GMST from ERA20C and ACCESS-CM2 on one plot:
ACCESS_CM2_GMST.plot(label='ACCESS-CM2')
ERA20C_GMST.plot(label='ERA20C')
plt.title('Global Mean Surface Temperature')
plt.legend() # Add a legend so that we can identify which curve is which (this uses the labels defined in the plot commands)

Both curves appear to have similar features. However, the ACCESS-CM2 GMST is several 10th's of a degree colder than ERA20C on average. There are a few possible reasons for this. One is that the model and/or observations may not represent the same regions. For example, due to its low resolution the model may not capture high altitude mountaneous regions (which are usually cold), or the observations may not cover the polar regions.

We are more interested in how these products represent *changes* in GMST. These changes are much more directly comparable.

To focus on GMST *changes*, we can subtract the average GMST within a pre-defined *baseline* period for each product. This next code block does this calculation using a baseline between 1900-1950:

In [0]:
# define baseline period
bline_per = [1900,1950] 

# Calculate mean GMST over the baseline period for each product:
ACCESS_CM2_GMST_baseline = ACCESS_CM2_GMST.sel(year=slice(bline_per[0],bline_per[1])).mean('year')
ERA20C_GMST_baseline = ERA20C_GMST.sel(year=slice(bline_per[0],bline_per[1])).mean('year')

# Subtract the baseline GMST to produce a GMST change ("deltaGMST") for each product:
ACCESS_CM2_deltaGMST = ACCESS_CM2_GMST-ACCESS_CM2_GMST_baseline
ERA20C_deltaGMST = ERA20C_GMST - ERA20C_GMST_baseline

In [0]:
# Plot both curves on one plot:
ACCESS_CM2_deltaGMST.plot(label='ACCESS-CM2')
ERA20C_deltaGMST.plot(label='ERA20C')

plt.title('Global Mean Surface Temperature [1900-1950 baseline]')
plt.legend()
plt.grid() # Add grid lines

That now looks better. This plot shows that the ACCESS-CM2 model does a reasonably good job of representing the overall warming of the GMST compared to the observations. This warming has reached about 0.8-0.9 degrees celsius (compared to the 1900-1950 baseline period) by the end of the simulation (2014). 

Assessment question:

17. <font color='red'> What are some possible reasons for the remaining discrepancies between the model and observations? To help you with your answer, load the data for the second ensemble member of the ACCESS-CM2 model (from the file `/course/data/ACCESS-CM2-E2_hist_data.nc`). Calculate the GMST change in ACCESS-CM2-E2 and add it to the plot above. Include this figure in your final report. Discuss how robust the differences between the modelled and observed GMST are. What are some potential causes for the year-to-year variability? **[6 marks]** </font>

Bonus question (for no marks): Can you identify any year-to-year features that are consistent between ACCESS-CM2 and ACCESS-CM2-E2 (hint: the image in `images/VolcanoList.png` contains a list of the biggest volcanic eruptions of the recent past)?

## Spatial structure of temperature and sea level changes

As a final exercise in this section, we look at the spatial structure of the SAT changes. 

The following code block calculates the longitude-latitude structure of the SAT during the last 10 years of the simulation (2004-2014) compared to the baseline 1900-1950 period.

In [0]:
# Calculate SAT averaged over the baseline and modern periods:
SAT_baseline = ACCESS_CM2_hist.SAT.sel(year=slice(bline_per[0],bline_per[1])).mean('year')
SAT_modern = ACCESS_CM2_hist.SAT.sel(year=slice(2004,2014)).mean('year')

# calculate the difference:
DIFF = SAT_modern-SAT_baseline

# Plot it:
DIFF.plot(cmap='RdBu_r',vmin=-5.,vmax=5.)
plt.title('ACCESS-CM2 Historical SAT in 2004-2014 minus SAT in 1900-1950')

Assessment questions:

18. <font color='red'> Explain why the high-latitude northern hemisphere (Arctic) shows a larger warming than the mid-latitudes and tropics. To help with your answer, calculate and plot the change in albedo (`SWupTOA/SWdownTOA`) over the same periods (2004-2014 minus baseline) as shown above. Include this plot in your report. Why might the Antarctic be different? **[4 marks]** </font>

19. <font color='red'> Do you notice any differences in the rate of warming of land vs. ocean regions? Why? **[2 marks]** </font>

20. <font color='red'> Plot the change in sea level (the variable `SL`) over the same period. Include this plot in your report. Describe the changes in sea level that you see and explain what might be driving them. **[4 marks]** </font>

# Bonus material:

If you're interested, please continue to explore the data that you've been given. For example, you could look at changes over different time periods.

You can also look in more detail at changes in a single location. For example, the next code block plots time series of the air temperature near Sydney according to the observations (ERA20C) and model (ACCESS-CM2):

In [0]:
# Sydney longitude and latitude:
lon = 151.2
lat = -33.8

# Load SAT at this location using xarray's .sel command:
ACCESS_CM2_SAT_Sydney = ACCESS_CM2_hist.SAT.sel(longitude=lon,latitude=lat,method='nearest')
ERA20C_SAT_Sydney = ERA20C_hist.SAT.sel(longitude=lon,latitude=lat,method='nearest')

# Plot:
ACCESS_CM2_SAT_Sydney.plot(label='ACCESS-CM2')
ERA20C_SAT_Sydney.plot(label='ERA20C')
plt.title('SAT changes near Sydney')
plt.legend()

According to both the model and observations, the Sydney SAT has warmed by about 1C over the historical period.